<a href="https://colab.research.google.com/github/hdpark1208/StudyCode/blob/main/CryptoWorkout_Week10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSA 실습

## RSA 알고리즘
1. 밥: 비밀 소수 $p, q$ 선택, $n=pq$ 계산
2. 밥: $gcd(e, (p-1)(q-1))=1$인 $e$ 선택
3. 밥: $de \equiv 1 \pmod{(p-1)(q-1)}$인 $d$ 계산
4. 밥: $n, e$ 공개 $p, q, d$ 비밀유지
5. 앨리스: 메시지 $m$을 $c \equiv m^e \pmod{n}$로 암호화 후, $c$를 밥에게 전달
6. 밥: $m \equiv c^d \pmod{n}$로 메시지 복호화

RSA 실제 사용을 위한 자료 링크: 

 - pycryptodome 공식문서
https://pycryptodome.readthedocs.io/en/latest/src/public_key/rsa.html
 - StackOverFlow 질의응답 : https://stackoverflow.com/questions/21327491/using-pycrypto-how-to-import-a-rsa-public-key-and-use-it-to-encrypt-a-string 

- 한글 블로그 https://comdoc.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%95%94%ED%98%B8%ED%99%94

RSA 알고리즘을 위에 예전 실습(5주차)에서 만들었던 코드들을 몇개 가져왔습니다.

In [ ]:
def GCD(a,b):
  if b == 0:
    return a       
  return GCD(b, a%b) 

def Extended_GCD(a,b):
  if b == 0:
    return a, 1, 0        #  GCD는 양수이므로 절대값 명령어 abs()를 사용
  gcd, x1, y1 = Extended_GCD(b, a%b) 
  x = y1
  y = x1 - (a//b)*y1 
#  print("{}={}*({})+{}*({})".format(gcd, a, x, b, y))  
# 나눗셈 정리를 눈으로 보고 싶으면 사용
  return gcd, x, y

def Modular_inv(a, n):
  if GCD(a, n) != 1:
    print("{}와 {}은 서로 소가 아닙니다".format(a, n))
    return
  return (Extended_GCD(a, n)[1] % n)

def Num_to_Binary(m):
  l = []
  q = m
  while q > 0:
    r = q % 2
    q = q // 2
    l = l+[r]
  return l

def Modular_power(a, m, n):
  l = Num_to_Binary(m)[::-1]
  x = 1
  for i in l:
    x = (x**2) % n
    if i==1:
      x = (x*a) % n
  return x

## RSA 예제
두 소수 `p=885320963`, `q=238855417`를 이용해서 RSA 알고리즘을 확인해봅시다.

In [ ]:
p = 885320963
q = 238855417
n = p*q
print(n)
e = 9007
GCD(e, (p-1)*(q-1)) # e는 (p-1)*(q-1)과 서로소

211463707796206571


1

공개키 $(n, e)$를 이용하여 메시지 $m$에 대해 $c \equiv m^e \pmod n$을 계산하면, 암호문 $c$를 얻을 수 있습니다.

In [ ]:
m = 30120
c = Modular_power(m, e, n)
print(c)

113535859035722866


d는 법 (p-1)*(q-1)에 대해 e의 역수입니다.

In [ ]:
d = Modular_inv(e, (p-1)*(q-1))
print(d)

116402471153538991


원래 메시지(평문)은 $c^d \pmod n$으로 복호화 하면 됩니다.

In [ ]:
Modular_power(c, d, n)

30120

$e=65537=2^{16}+1$을 사용하는 경우가 많습니다.
그 이유는 2진법으로 매우 간단하기 때문에 연산횟수가 상대적으로 적습니다.

In [ ]:
Num_to_Binary(65537)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

## 큰 소수 만드는 방법

목록 `first_primes_list`안의 작은 소수들로 나눠지는지 여부를 먼저 체크하고
강의에서 소개한 Miller-Rabin 소수판정법을 여러번 실행해서 주어진 범위의(n 비트) "매우 가능성이 높은" 소수를 찾아줍니다. (이 코드에서는 20번 실행합니다)
출처: [How to generate Large Prime numbers for RSA Algorithm](https://www.geeksforgeeks.org/how-to-generate-large-prime-numbers-for-rsa-algorithm/ )

In [ ]:
# Large Prime Generation for RSA
import random

# Pre generated primes
first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
					31, 37, 41, 43, 47, 53, 59, 61, 67,
					71, 73, 79, 83, 89, 97, 101, 103,
					107, 109, 113, 127, 131, 137, 139,
					149, 151, 157, 163, 167, 173, 179,
					181, 191, 193, 197, 199, 211, 223,
					227, 229, 233, 239, 241, 251, 257,
					263, 269, 271, 277, 281, 283, 293,
					307, 311, 313, 317, 331, 337, 347, 349]

def nBitRandom(n):
	return random.randrange(2**(n-1)+1, 2**n - 1)

def getLowLevelPrime(n):
	'''Generate a prime candidate divisible
	by first primes'''
	while True:
		# Obtain a random number
		pc = nBitRandom(n)

		# Test divisibility by pre-generated
		# primes
		for divisor in first_primes_list:
			if pc % divisor == 0 and divisor**2 <= pc:
				break
		else: return pc

def isMillerRabinPassed(mrc):
	'''Run 20 iterations of Rabin Miller Primality test'''
	maxDivisionsByTwo = 0
	ec = mrc-1
	while ec % 2 == 0:
		ec >>= 1
		maxDivisionsByTwo += 1
	assert(2**maxDivisionsByTwo * ec == mrc-1)

	def trialComposite(round_tester):
		if pow(round_tester, ec, mrc) == 1:
			return False
		for i in range(maxDivisionsByTwo):
			if pow(round_tester, 2**i * ec, mrc) == mrc-1:
				return False
		return True

	# Set number of trials here
	numberOfRabinTrials = 20
	for i in range(numberOfRabinTrials):
		round_tester = random.randrange(2, mrc)
		if trialComposite(round_tester):
			return False
	return True

if __name__ == '__main__':
	while True:
		n = 64
		prime_candidate = getLowLevelPrime(n)
		if not isMillerRabinPassed(prime_candidate):
			continue
		else:
			print(n, "bit prime is: \n", prime_candidate)
			break


64 bit prime is: 
 10777577063589369659


또다른 64비트 소수를 만들어봅시다.

In [ ]:
while True:
		n = 64
		prime_candidate = getLowLevelPrime(n)
		if not isMillerRabinPassed(prime_candidate):
			continue
		else:
			print(n, "bit prime is: \n", prime_candidate)
			break

64 bit prime is: 
 10278178819364820487


일반적으로 소인수분해는 매우 오랜 시간이 걸립니다.

In [ ]:
from sympy.ntheory import factorint # sympy 라이브러리의 소인수분해함수를 이용
p = 12753173587119043723
q = 11293880751381704837
n = p*q
print(n)
#factorint(n)   # 64비트 소수 2개의 곱은 37초 정도에 소인수분해됨


144032821694593337494021055856883588151


## 실습문제

1. 암호문 5859가 공개키 $n = 11413, e = 7647$을 이용해서 $n$이 $101 \cdot 113$으로 소인수분해되는 것을 이용하여, 원래의 평문을 구하여라. 

2. [손으로 풀기] 암호문 $c$가 $c \equiv m^3 \pmod{101}$로 계산되었다고 할때, 
복호화 방법을 알아내어라. (주의: 101은 소수이다)

3. [손으로 풀기] 지수 $e = 1$ 또는 $e = 2$는 RSA 암호시스템에서 사용되지 않아야 한다. 
그 이유는?

4. 공개키 `n = 718548065973745507,
e = 3449`를 이용해서 암호문 `c=543546506135745129`을 얻었다. 
원래 메시지를 구하여라. (힌트: sympy 라이브러리의 factorint() 명령어)


In [ ]:
c=5859
p, q =101,113
n = p*q
e = 7647
d=Modular_inv(e,n)
m=Modular_power(c,d,n)
print(m): 6566

In [ ]:
from sympy import factorint
n = 718548065973745507
factor_dict = factorint(n) 
      
print(factor_dict)
c=543546506135745129
p=740876531
q=969862097
n=p*q
e=3449
d=Modular_inv(e,n)
m=Modular_power(c,d,n)
print(m)



{740876531: 1, 969862097: 1}
63258829968764216


6566


## 실습문제 2
1. 위의 코드들을 참고하여 **자신만의** 128비트 정도 크기의 공개키 `n`과 암호화 지수 `e`를 만들어서 안내한 공유문서에 업로드: 링크: https://docs.google.com/spreadsheets/d/1-invVKwUbGlwrm1h1w4eY8yCGUzPMvS89g3ipnHOF2Q/edit?usp=sharing 
2. 자신에게 **암호화된 전달문**을 받을 사람은, 그것을 해독하여 그 숫자 뒤에 **자신의 학번 끝자리 세 수**를 붙여서 
  
  1. 만약 15자리이면, 담당교수 혹은 담당조교의 공개키로 암호화하여 전달
  2. 아직 15자리 미만이면, 다시 (아직 전달문을 받지 않은 사람 중) 전달하고 싶은 한 명에게 그 사람의 공개키를 이용, 암호화하여 전달(공유문서의 암호화된 전달문 칸에 업로드)
  

36

352